In [4]:
# Pivoting Data with Polars
import polars as pl

In [ ]:
# Load World Bank Population data from CSV file
world_bank_pop = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/world_bank_pop.csv", null_values=["NA", ''])

print("World Bank Population Dataset Shape:", world_bank_pop.shape)
print("\nColumn names:", world_bank_pop.columns)
print("\nFirst few rows:")
world_bank_pop.head()

World Bank Population Dataset Shape: (1064, 21)

Column names: ['rownames', 'country', 'indicator', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']

First few rows:


rownames,country,indicator,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""ABW""","""SP.URB.TOTL""",41625.0,42025.0,42194.0,42277.0,42317.0,42399.0,42555.0,42729.0,42906.0,43079.0,43206.0,43493.0,43864.0,44228.0,44588.0,44943.0,45297.0,45648.0
2,"""ABW""","""SP.URB.GROW""",1.664222,0.956373,0.401335,0.196517,0.094569,0.193588,0.367258,0.408049,0.413383,0.402396,0.294374,0.662063,0.849393,0.826413,0.810669,0.793026,0.784578,0.771899
3,"""ABW""","""SP.POP.TOTL""",89101.0,90691.0,91781.0,92701.0,93540.0,94483.0,95606.0,96787.0,97996.0,99212.0,100341.0,101288.0,102112.0,102880.0,103594.0,104257.0,104874.0,105439.0
4,"""ABW""","""SP.POP.GROW""",2.539234,1.768757,1.194718,0.997396,0.900989,1.003077,1.181566,1.227711,1.241397,1.233231,1.131541,0.939356,0.810231,0.749301,0.691615,0.637959,0.590062,0.537296
5,"""AFE""","""SP.URB.TOTL""",1.15551653e8,1.19775502e8,1.24227507e8,1.28833965e8,1.33647466e8,1.38745587e8,1.4402674e8,1.49231302e8,1.55383774e8,1.61776165e8,1.68456076e8,1.75415651e8,1.82558745e8,1.90108665e8,1.98073341e8,2.06556291e8,2.15083329e8,2.23732118e8


In [6]:
## Is this useful?

# Sometimes you have to work with wide data. In wide data, you will have many instances 
# of the same metric in the same row. How could you, for example, take a sum of ALL total population data?

# This would be cumbersome in wide format - you'd have to list all year columns
total_pop = world_bank_pop.filter(pl.col("indicator") == "SP.POP.TOTL")
population_sum = total_pop.select([pl.col(str(year)).sum() for year in range(2000, 2018)])

print("The wide format makes it difficult to aggregate across all years")

The wide format makes it difficult to aggregate across all years


## Going from wide to narrow datasets in order to summarize

We need to make sure that each yearly metric has its own row instead of being combined into one row.

**Pivoting data from "wide" to "long"** - Think about this like an open door. We want to "swing" the door shut by placing the values of certain column headers into a single column. 

In Polars, we use `unpivot()`:
- `index`: Columns to keep as identifiers (the "hinge")
- `on`: Columns to unpivot (the "door" we're closing)
- `variable_name`: Name for the new column containing former column names
- `value_name`: Name for the new column containing the values

In [12]:
# Pivot from wide to long format using unpivot()

# Get all year columns (all columns except country and indicator)
year_columns = [col for col in world_bank_pop.columns if col not in ["country", "indicator", "rownames"]]

# Unpivot the dataframe from wide to long
long_dataset = world_bank_pop.unpivot(
    index=["country", "indicator"],     # These are the "hinge" columns
    on=year_columns,                     # These are the columns to unpivot
    variable_name="year",                # Name for the new column with year values
    value_name="metric"                  # Name for the new column with metric values
).with_columns(pl.col("year").cast(pl.Int64()))

print(f"Long dataset shape: {long_dataset.shape}")
print("\nFirst 10 rows of long dataset:")
long_dataset.head(10)

Long dataset shape: (19152, 4)

First 10 rows of long dataset:


country,indicator,year,metric
str,str,i64,f64
"""ABW""","""SP.URB.TOTL""",2000,41625.0
"""ABW""","""SP.URB.GROW""",2000,1.664222
"""ABW""","""SP.POP.TOTL""",2000,89101.0
"""ABW""","""SP.POP.GROW""",2000,2.539234
"""AFE""","""SP.URB.TOTL""",2000,1.15551653e8
"""AFE""","""SP.URB.GROW""",2000,3.602262
"""AFE""","""SP.POP.TOTL""",2000,4.01600588e8
"""AFE""","""SP.POP.GROW""",2000,2.583579
"""AFG""","""SP.URB.TOTL""",2000,4.3147e6


### Now we can better aggregate metrics across years

Find the average total population for each country

In [13]:
# Calculate average total population for each country

avg_pop_per_country = (
    long_dataset
    .filter(pl.col("indicator") == "SP.POP.TOTL")  # Filter for total population
    .group_by("country")                            # Group by country
    .agg(pl.col("metric").mean().alias("avg_pop"))  # Calculate mean, handling nulls
)

print("Average population per country:")
avg_pop_per_country

Average population per country:


country,avg_pop
str,f64
"""TSA""",1.6212e9
"""QAT""",1506079.5
"""BHS""",364164.666667
"""EUU""",4.3860e8
"""ECA""",3.7928e8
…,…
"""NAC""",3.3812e8
"""BGR""",7.5095e6
"""SDN""",3.2692e7


## Aggregating columns with multiple measures

When working with data, it's best to have exactly one definition for the metrics of a column. In the example below, there are metrics for urban population, urban population growth, total population, and total population growth.

The units of the data within the metric column are different - the population metrics are in the units of people whereas the growth metrics are in the units of percents.

In [ ]:
# Trying to sum all metrics doesn't make sense when they have different units
metrics_tot = long_dataset.select(pl.col("metric").sum())

print("Sum of all metrics (meaningless number due to mixed units):")
print(f"Total: {metrics_tot.item():,.2f}")
print("\nThis combines population counts (in millions) with growth rates (in percents)!")

### Creating one column per metric (narrow to wide)

It is better to pivot the data from narrow to wide to ensure that every column represents exactly one definition. Consider a long dataset like a closed door that we want to open.

In Polars, we use `pivot()`:
- `index`: Columns that identify each observation (the "hinge")  
- `columns`: Column whose values become new column names (the "door" we're opening)
- `values`: Column containing the values to fill the new columns

In [14]:
# Pivot from long to wide format

wider_dataset = long_dataset.pivot(
    index=["country", "year"],     # These columns identify each observation (the "hinge")
    columns="indicator",            # Values from this column become new column names (the "door")
    values="metric"                 # Values to fill the new columns
)

print(f"Wider dataset shape: {wider_dataset.shape}")
print("\nNow each indicator has its own column:")
print("Columns:", wider_dataset.columns)
print("\nFirst 10 rows:")
wider_dataset.head(10)

Wider dataset shape: (4788, 6)

Now each indicator has its own column:
Columns: ['country', 'year', 'SP.URB.TOTL', 'SP.URB.GROW', 'SP.POP.TOTL', 'SP.POP.GROW']

First 10 rows:


/var/folders/cq/gb4n_zwd509fb_pw_5ws7xgh0000gn/T/ipykernel_25899/2810382385.py:3: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  wider_dataset = long_dataset.pivot(


country,year,SP.URB.TOTL,SP.URB.GROW,SP.POP.TOTL,SP.POP.GROW
str,i64,f64,f64,f64,f64
"""ABW""",2000,41625.0,1.664222,89101.0,2.539234
"""AFE""",2000,1.15551653e8,3.602262,4.01600588e8,2.583579
"""AFG""",2000,4.3147e6,1.861377,1.9542982e7,1.443803
"""AFW""",2000,9.527289e7,4.145189,2.69611898e8,2.7496
"""AGO""",2000,8.211294e6,5.64867,1.6394062e7,3.244121
"""ALB""",2000,1.289391e6,0.742479,3.089027e6,-0.637357
"""AND""",2000,61070.0,0.377328,66097.0,0.67096
"""ARB""",2000,1.52305719e8,2.761375,2.87065982e8,2.285934
"""ARE""",2000,2.627996e6,6.112729,3.275333e6,5.580387


## Additional Dataset Examples

### Billboard Dataset Example

In [ ]:
# Load the actual billboard dataset from CSV
billboard = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/billboard.csv")

print(f"Billboard dataset shape: {billboard.shape}")
print("\nColumn names:", billboard.columns[:10])  # Show first 10 column names
print("\nFirst few rows with selected columns:")
# Show first few columns to see the structure
billboard.select(billboard.columns[:6]).head()

Billboard dataset shape: (317, 80)

Column names: ['rownames', 'artist', 'track', 'date.entered', 'wk1', 'wk2', 'wk3', 'wk4', 'wk5', 'wk6']

First few rows with selected columns:


rownames,artist,track,date.entered,wk1,wk2
i64,str,str,str,i64,i64
1,"""2 Pac""","""Baby Don't Cry (Keep...""","""2000-02-26""",87,82
2,"""2Ge+her""","""The Hardest Part Of ...""","""2000-09-02""",91,87
3,"""3 Doors Down""","""Kryptonite""","""2000-04-08""",81,70
4,"""3 Doors Down""","""Loser""","""2000-10-21""",76,76
5,"""504 Boyz""","""Wobble Wobble""","""2000-04-15""",57,34


In [16]:
# Transform billboard data from wide to long format

# Get all week columns (assuming they start with 'wk')
week_columns = [col for col in billboard.columns if col.startswith('wk')]

# Get the ID columns (everything except week columns)
id_columns = [col for col in billboard.columns if not col.startswith('wk')]

longer_billboard = (
    billboard
    .unpivot(
        index=id_columns,                              # Keep these columns
        on=week_columns,                                # Unpivot week columns
        variable_name="week_num",                      # Name for week column
        value_name="chart_position"                    # Name for position values
    )
    .filter(pl.col("chart_position").is_not_null())   # Remove nulls (songs off chart)
)

print(f"Long billboard dataset shape: {longer_billboard.shape}")
print("\nFirst 10 rows showing chart positions over weeks:")
longer_billboard.head(10)

Long billboard dataset shape: (5307, 6)

First 10 rows showing chart positions over weeks:


rownames,artist,track,date.entered,week_num,chart_position
i64,str,str,str,str,str
1,"""2 Pac""","""Baby Don't Cry (Keep...""","""2000-02-26""","""wk1""","""87"""
2,"""2Ge+her""","""The Hardest Part Of ...""","""2000-09-02""","""wk1""","""91"""
3,"""3 Doors Down""","""Kryptonite""","""2000-04-08""","""wk1""","""81"""
4,"""3 Doors Down""","""Loser""","""2000-10-21""","""wk1""","""76"""
5,"""504 Boyz""","""Wobble Wobble""","""2000-04-15""","""wk1""","""57"""
6,"""98^0""","""Give Me Just One Nig...""","""2000-08-19""","""wk1""","""51"""
7,"""A*Teens""","""Dancing Queen""","""2000-07-08""","""wk1""","""97"""
8,"""Aaliyah""","""I Don't Wanna""","""2000-01-29""","""wk1""","""84"""
9,"""Aaliyah""","""Try Again""","""2000-03-18""","""wk1""","""59"""


### US Rent Income Dataset Example

In [ ]:
# Load the actual US rent/income dataset from CSV
us_rent_income = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/us_rent_income.csv")

print("US Rent/Income dataset shape:", us_rent_income.shape)
print("\nColumn names:", us_rent_income.columns)
print("\nDataset (long format):")
us_rent_income.head(10)

US Rent/Income dataset shape: (104, 6)

Column names: ['rownames', 'GEOID', 'NAME', 'variable', 'estimate', 'moe']

Dataset (long format):


rownames,GEOID,NAME,variable,estimate,moe
i64,i64,str,str,i64,i64
1,1,"""Alabama""","""income""",24476,136
2,1,"""Alabama""","""rent""",747,3
3,2,"""Alaska""","""income""",32940,508
4,2,"""Alaska""","""rent""",1200,13
5,4,"""Arizona""","""income""",27517,148
6,4,"""Arizona""","""rent""",972,4
7,5,"""Arkansas""","""income""",23789,165
8,5,"""Arkansas""","""rent""",709,5
9,6,"""California""","""income""",29454,109


In [22]:
# Pivot US rent/income data to wide format
# This creates separate columns for rent_estimate, rent_moe, income_estimate, income_moe

# First, we need to reshape the data to have separate columns for estimate and moe
# In polars, we can do this by pivoting twice and then joining, or by melting and pivoting

# Method: Create separate dataframes for estimate and moe, then join
wider_rent_income = us_rent_income.pivot(
    index=["GEOID", "NAME"],
    on="variable",
    values=["estimate", "moe"]
)

print("US Rent/Income dataset (wide format):")
print("Now each metric (rent/income) has separate columns for estimate and margin of error:")
wider_rent_income

US Rent/Income dataset (wide format):
Now each metric (rent/income) has separate columns for estimate and margin of error:


GEOID,NAME,estimate_income,estimate_rent,moe_income,moe_rent
i64,str,i64,i64,i64,i64
1,"""Alabama""",24476,747,136,3
2,"""Alaska""",32940,1200,508,13
4,"""Arizona""",27517,972,148,4
5,"""Arkansas""",23789,709,165,5
6,"""California""",29454,1358,109,3
…,…,…,…,…,…
53,"""Washington""",32318,1120,113,4
54,"""West Virginia""",23707,681,203,6
55,"""Wisconsin""",29868,813,135,3
